In [2]:
import sys
import codecs
import pandas as pd
import matplotlib.pyplot as plt


# shops = pd.read_csv('shops.txt', delimiter='\t')
# reviews = pd.read_csv('reviews.txt', delimiter='\t')

shops = pd.read_csv('as.txt', delimiter='\t')
reviews = pd.read_csv('ar.txt', delimiter='\t')

print(reviews)

    店鋪ID                  メニュー    分類   スープ  点数          コメント
0  34567         豚骨貝介のワイルドWスープ  ラーメン  豚骨魚介  80           みたか
1   2234  がんメン 50円 + 替玉（無料券利用）  ラーメン    豚骨  70   安心の<br />替玉
2    392                 いのしし麺  ラーメン    醤油  83            う！
3      1                   いけ麺   つけ麺    醤油  87         つi-to
4     91                    そば  ラーメン    醤油  80  ですです。<br />よ
